## Описание данных
Набор данных находится в файле /datasets/insurance.csv.
Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
Целевой признак: количество страховых выплат клиенту за последние 5 лет.

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

##  Содержание проекта

<a href='#step1'>1.Загрузка данных</a>

<a href='#step2'>2.Умножение матриц</a>

<a href='#step3'>3.Алгоритм преобразования</a>

<a href='#step4'>4.Проверка алгоритма</a>

## 1. Загрузка данных <a id='step1'></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
from numpy.linalg import inv

Загрузим данные

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Узнаем какие варианты целевого признака представлены.

In [4]:
data['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

Проверим, присутвуют ли пропуски и дубликаты.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
data.duplicated().sum()

153

In [7]:
data = data.drop_duplicates().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
Пол                  4847 non-null int64
Возраст              4847 non-null float64
Зарплата             4847 non-null float64
Члены семьи          4847 non-null int64
Страховые выплаты    4847 non-null int64
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


### Вывод

Данные загружены и просмотрены на дубликаты и пропуски

## 2. Умножение матриц <a id='step2'></a>

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится. Предсказания останутся такими же.

**Обоснование:** Если заметить исходную матрицу X на преобразованную матрицу XP в формуле предсказания и формуле обучения, матрица P сократится до единичной.$$
a = XP ((X P)^T X P)^{-1} (X P)^T y = XP (P^T X^T X P)^{-1} P^T X^T y = 𝑋 (X^T X)^{-1} X^T y =Xw
$$

## 3. Алгоритм преобразования <a id='step3'></a>

**Алгоритм**

Необходимо умножить матрицу признаком умножить на сгенерированную обратимую матрицу.

**Обоснование**

Поскольку после после умножения матрицы признаков предсказанияне изменятся, качество модели останется таким же.

## 4. Проверка алгоритма <a id='step4'></a>

Узнаем качество модели на исходных данных.

In [8]:
target = data['Страховые выплаты']
features = data.drop('Страховые выплаты', axis=1)
features_valid, features_test, target_valid, target_test = train_test_split(features, target, \
                                                                            test_size=0.40, random_state=12345)

In [9]:
model = LinearRegression()
model.fit(features_valid,target_valid)
predictions = model.predict(features_test)
print ('R2 =', r2_score(target_test,predictions))

R2 = 0.4272661343811639


Теперь преобразуем данные через умножение на обратимую матрицу.

In [10]:
P = np.random.normal(loc=0.0, scale=1.0, size=[5,5])
i = 0
while i == 0:
    try:
        np.linalg.inv(P)
        i = 1
    except:
        i = 0

In [11]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
XP = np.dot(X,P)

new_target = data['Страховые выплаты']
new_features = np.delete(XP, 0, axis=1)
new_features_valid, new_features_test, new_target_valid, \
new_target_test = train_test_split(new_features, new_target, test_size=0.40, random_state=12345)

Проверим качество модели на преобразованных данных.

In [12]:
new_model = LinearRegression()
new_model.fit(new_features_valid,new_target_valid)
new_predictions = model.predict(new_features_test)
print ('R2 =', r2_score(new_target_test,predictions))

R2 = 0.4272661343811639


### Вывод

Качество моделей не ухудшилось после преобразований.